In [2]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os
import random
import sys

In [7]:
_RANDOM_SEED=0
NUM_TEST=100
DATASET_DIR='D:/Tensorflow/captcha/images/'
TFRECORD_DIR='D:/Tensorflow/captcha/'
def data_exists(dataset_dir):
    for split_name in ['train','test']:
        output_filename=os.path.join(dataset_dir,split_name+'.tfrecords')
        if not tf.gfile.Exists(output_filename):
            return False
    return True
def get_file_and_classnames(dataset_dir):
    photo_filenames=[]
    for filename in os.listdir(dataset_dir):
        path=os.path.join(dataset_dir,filename)
        photo_filenames.append(path)    
    return photo_filenames
def int64_feature(values):
    if not isinstance(values,(tuple,list)):
        values=[values]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

def bytes_feature(values):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[values]))
def image_to_tfexample(image_data,labels):
    return tf.train.Example(features=tf.train.Features(feature={
        'image':bytes_feature(image_data),
        'label0': int64_feature(labels[0]),
        'label1': int64_feature(labels[1]),
        'label2': int64_feature(labels[2]),
        'label3': int64_feature(labels[3]),
    }))
def _convert_dataset(split_name,filenames,dataset_dir):
    assert split_name in ['train','test']
    with tf.Session() as sess:
        output_filename=os.path.join(TFRECORD_DIR,split_name+'.tfrecords')
        with tf.python_io.TFRecordWriter(output_filename) as tfrecord_writer:
            for i,filename in enumerate(filenames):
                try:
                    sys.stdout.write('\r>> converting image %d/%d'% (i+1,len(filenames)))
                    sys.stdout.flush()
                    image_data=Image.open(filename)
                    image_data=image_data.resize((224,224))
                    image_data=np.array(image_data.convert('L'))
                    image_data=image_data.tobytes()
                    labels=filename.split('/')[-1][0:4]
                    num_labels=[]
                    for j in range(4):
                        num_labels.append(int(labels[j]))
                    example=image_to_tfexample(image_data,num_labels)
                    tfrecord_writer.write(example.SerializeToString())
                except IOError as e:
                    print('could not read:',filename)
                    print('Error :',e)
                    print('skip it\n')
        sys.stdout.write('\n')
        sys.stdout.flush()           
                    
                    
if __name__=='__main__':
    if data_exists(TFRECORD_DIR):
        print('file exsits')
    else:
        photo_filenames=get_file_and_classnames(DATASET_DIR)
        random.seed(_RANDOM_SEED)
        random.shuffle(photo_filenames)
        training_filenames=photo_filenames[NUM_TEST:]
        testing_filenames=photo_filenames[:NUM_TEST]
        _convert_dataset('train',training_filenames,DATASET_DIR)
        _convert_dataset('test',testing_filenames,DATASET_DIR)
        print('done')
        

>> converting image 6216/6216
>> converting image 100/100
done
